### Check 1
- Check if fforum have question and answer
- Replace bad characters in question and answer
- Check if forum have more than 1 view
- Check if question is longer than 19 characters
- Check if answer is longer than 50 characters
- Check if user that answered the question is not anonymous

In [ ]:
import json
from tqdm import tqdm

with open("../scraper_medovernet/merged_data.json") as f:
    data = json.load(f)


new_data_meodver = []

check_answer = 0
check_user = 0
combine_error = 0
len_answer_question = 0
views = 0
bad_chars = 0


with tqdm(total=len(data), desc="Checking ...") as pbar:
    for ans in data:
        forum_data = ans["forum_data"]

        if len(forum_data) > 1:
            question = forum_data[0]
            answer = forum_data[1]

            if any(char in question for char in ["\n", '"', "-"]) or any(
                char in answer for char in ["\n", '"', "-"]
            ):
                bad_chars += 1 

            question_content = (
                question["content"]
                .replace("\n", " ")
                .replace('"', "")
                .replace("-", " ")
                .replace("_", " ")
            )
            answer_content = (
                answer["content"]
                .replace("\n", " ")
                .replace('"', "")
                .replace("-", " ")
                .replace("_", " ")
            )

            if int(question["views"].replace(".", "")) > 1:
                if len(answer_content) > 19 and len(question_content) > 50:
                    if '<a href="https://med.over.net/members/' in answer["user_html"]:
                        new_data_meodver.append(
                            {
                                "category": ans["category"],
                                "forum": ans["forum"],
                                "posts": ans["posts"],
                                "subforum": ans["subforum"],
                                "forum_name": ans["forum_name"],
                                "forum_link": ans["forum_link"],
                                "forum_data": [
                                    {
                                        "user": question["user"],
                                        "date": question["date"],
                                        "content": question_content,
                                        "views": question["views"],
                                        "user_html": question["user_html"],
                                    },
                                    {
                                        "user": answer["user"],
                                        "date": answer["date"],
                                        "content": answer_content,
                                        "views": answer["views"],
                                        "user_html": answer["user_html"],
                                    },
                                ],
                            }
                        )
                    else:
                        check_user += 1
                else:
                    len_answer_question += 1
            else:
                views += 1
        else:
            check_answer += 1

        combine_error = check_answer + check_user + len_answer_question + views

        pbar.set_postfix(
            {
                "without answer": check_answer,
                "views": views,
                "len_answer_question": len_answer_question,
                "anonymous answer": check_user,
                "bad_chars": bad_chars,
                "combine_error": combine_error,
            }
        )

        pbar.update(1)

In [58]:
with open("med-over-net-data.json", "w", encoding="utf-8") as f:
    json.dump(new_data_meodver, f, ensure_ascii=False, indent=4)

### Check 2
- Checking grammar mistakes in question and answer

In [ ]:
from tqdm import tqdm
import language_tool_python
import json

tool = language_tool_python.LanguageTool("sl")
new_data_grammar_medover = []
bad_sentence = 0
bad_sentence_data = []

with open("med-over-net-data.json") as f:
    data = json.load(f)

with tqdm(total=len(data), desc="Checking profanity") as pbar:
    for ans in data:
        question = ans["forum_data"][0]["content"]
        answer = ans["forum_data"][1]["content"]
        check_sentence_question = tool.check(question)
        check_sentence_answer = tool.check(answer)

        if len(check_sentence_question) <= 20 and len(check_sentence_answer) <= 20:
            new_data_grammar_medover.append(ans)
        else:
            bad_sentence_data.append(
                {
                    "url": ans["url"],
                    "question_err": check_sentence_question,
                    "answer_err": check_sentence_answer,
                    "question": question,
                    "answer": answer,
                }
            )
            bad_sentence += 1

        pbar.set_postfix({"bad sentence": bad_sentence})
        pbar.update(1)

tool.close()

In [ ]:
bad_sentence_data = sorted(
    bad_sentence_data, key=lambda x: len(x["question_err"]) + len(x["answer_err"]), reverse=True
)
bad_sentence_data[0]

In [ ]:
with open("med-over-net-data-grammar.json", "w", encoding="utf-8") as f:
    json.dump(new_data_grammar_medover, f, ensure_ascii=False, indent=4)

### Check 3
- Check if bad words are present in question and answer

In [ ]:
from translate import Translator
from profanity_check import predict
from tqdm import tqdm


def translate_to_slovenian_and_check_profanity(text, translator):
    translated_text = translator.translate(text)
    profani = predict([translated_text])[0]
    return True if profani == 1 else False


if __name__ == "__main__":
    new_data_not_profanity = []
    prof_count = 0
    translator = Translator(from_lang="sl", to_lang="en")

    with open("med-over-net-data.json") as f:
        data = json.load(f)

    with tqdm(total=len(data), desc="Checking profanity") as pbar:
        for ans in data:
            question = ans["forum_data"][0]["content"]
            answer = ans["forum_data"][1]["content"]
            check_profanity_question = translate_to_slovenian_and_check_profanity(question, translator)
            check_profanity_answer = translate_to_slovenian_and_check_profanity(answer, translator)

            if not check_profanity_question and not check_profanity_answer:
                new_data_not_profanity.append(ans)
            else:
                prof_count += 1

            pbar.set_postfix({"profanity": prof_count})
            pbar.update(1)

In [ ]:
with open("./final-data/med-over-net-data-final.json", "w", encoding="utf-8") as f:
    json.dump(new_data_not_profanity, f, ensure_ascii=False, indent=4)